In [1]:
import os

In [2]:
%pwd

'D:\\Python\\MLProjectsPW\\CreditCardFraud1\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'D:\\Python\\MLProjectsPW\\CreditCardFraud1'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    local_data_file:Path    

In [6]:
from CreditCardFraud.constants import *
from CreditCardFraud.utils.common import read_yaml , create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            local_data_file= config.local_data_file
        )

        return data_ingestion_config


In [8]:
from CreditCardFraud.logging import logger
from CreditCardFraud.utils.common import get_size
from dotenv import load_dotenv
from pymongo import MongoClient
import pandas as pd

In [9]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            load_dotenv()
            filename = self.config.local_data_file
            mongo_client_con_string =  os.getenv("pymongo_url")
            database_name ='demoDB'
            collection_name = 'CreditCardFraud'

            myclient = MongoClient(mongo_client_con_string)
            mydb = myclient[database_name]
            mycol = mydb[collection_name]


            mongo_docs = list(mycol.find())
            data = pd.DataFrame(mongo_docs)
            data.sort_values('_id').reset_index(drop=True).iloc[:,1:].to_csv(filename,index=False)
            logger.info(f"{filename} downloaded!")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")

In [10]:
try:
    config  = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
except Exception as e:
    raise e

[2023-08-11 19:48:35,358: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-11 19:48:35,367: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-11 19:48:35,370: INFO: common: creating directory at: artifacts]
[2023-08-11 19:48:35,373: INFO: common: creating directory at: artifacts/data_ingestion]
[2023-08-11 19:48:35,375: INFO: 2304682800: file already exists of size: ~88 KB]
